In [22]:
import pandas as pd
import seaborn as sns
import sys
import string
print('Original sys.path:', sys.path)

# Append a new directory to sys.path
sys.path.append('C:/Users/jp3g20/Desktop/data mining/summary-eval')

# Print the updated sys.path
print('Updated sys.path:', sys.path)
from summary_eval.data import summary_df, prompts_df
from summary_eval.settings import TRAIN_SIZE
from summary_eval.testing import cross_validate

merged_df = pd.merge(summary_df, prompts_df, on='prompt_id')

Original sys.path: ['c:\\Users\\jp3g20\\Desktop\\data mining\\summary-eval\\notebooks', 'C:\\Apps\\Anaconda3\\python311.zip', 'C:\\Apps\\Anaconda3\\DLLs', 'C:\\Apps\\Anaconda3\\Lib', 'C:\\Apps\\Anaconda3', 'c:\\Users\\jp3g20\\Desktop\\data mining\\summary-eval\\venv', '', 'c:\\Users\\jp3g20\\Desktop\\data mining\\summary-eval\\venv\\Lib\\site-packages', 'c:\\Users\\jp3g20\\Desktop\\data mining\\summary-eval\\venv\\Lib\\site-packages\\win32', 'c:\\Users\\jp3g20\\Desktop\\data mining\\summary-eval\\venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jp3g20\\Desktop\\data mining\\summary-eval\\venv\\Lib\\site-packages\\Pythonwin', 'C:/Users/jp3g20/Desktop/summary_eval/summary-eval', 'C:/Users/jp3g20/Desktop/data mining/summary-eval', 'C:/Users/jp3g20/Desktop/data mining/summary-eval', 'C:/Users/jp3g20/Desktop/data mining/summary-eval', 'C:/Users/jp3g20/Desktop/data mining/summary-eval', 'C:/Users/jp3g20/Desktop/data mining/summary-eval']
Updated sys.path: ['c:\\Users\\jp3g20\\Desktop\\data

In [23]:

import torch
print(torch.__version__)
from transformers import BertTokenizer, BertModel
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import HistGradientBoostingRegressor


2.3.0+cu118


In [24]:
import torch
import numpy as np
from transformers import BertTokenizer, BertModel

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained("bert-base-uncased")

# Function to calculate average embedding of non-stopwords using BERT
def calculate_average_bert_embedding(text):
    # Tokenize the text
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True, padding=True)

    # Get BERT outputs
    with torch.no_grad():
        outputs = bert_model(**inputs)
    
    # Extract hidden states from BERT outputs
    hidden_states = outputs.last_hidden_state
    
    # Average pooling over all tokens (excluding padding tokens)
    non_pad_mask = inputs["attention_mask"].unsqueeze(-1).expand(hidden_states.size())
    non_pad_hidden_states = hidden_states * non_pad_mask
    sum_hidden_states = torch.sum(non_pad_hidden_states, 1)
    sum_mask = non_pad_mask.sum(1)
    mean_pooled = sum_hidden_states / sum_mask
    
    # Convert tensor to numpy array
    avg_embedding = mean_pooled.squeeze().numpy()
    
    return avg_embedding

# Apply function to 'text', 'prompt_text', and 'prompt_question' columns
merged_df['text_embeddings'] = merged_df['text'].apply(calculate_average_bert_embedding)
merged_df['prompt_embeddings'] = merged_df['prompt_text'].apply(calculate_average_bert_embedding)
merged_df['prompt_question_embeddings'] = merged_df['prompt_question'].apply(calculate_average_bert_embedding)





c:\Users\jp3g20\Desktop\data mining\summary-eval\venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [25]:
merged_df.to_csv('temp_csv.csv')

In [34]:
import numpy as np
from sklearn.model_selection import KFold

n_folds = 10 

k_folds = KFold(n_splits=n_folds, shuffle=True, random_state=42)

X_text_embeddings = np.vstack(merged_df['text_embeddings'].values)
X_prompt_embeddings = np.vstack(merged_df['prompt_embeddings'].values)
X_prompt_question_embeddings = np.vstack(merged_df['prompt_question_embeddings'].values)


X_embeddings = np.hstack((X_text_embeddings, X_prompt_embeddings, X_prompt_question_embeddings))
y = merged_df[['content', 'wording']]
X_train, X_test, y_train, y_test = train_test_split(X_embeddings, y, test_size=0.2, random_state=42)
model = MultiOutputRegressor(HistGradientBoostingRegressor(random_state=0))

for train_i, test_i in k_folds.split(X_train):
    fold_train_X, fold_test_X = X_train[train_i], X_train[test_i]
    fold_train_y, fold_test_y = y_train.to_numpy()[train_i], y_train.to_numpy()[test_i]
    model.fit(fold_train_X, fold_train_y)





In [35]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)

mse = mean_squared_error(y_test, y_pred)

r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("R-squared (R^2) score:", r2)


Mean Absolute Error (MAE): 0.43415684788616127
Mean Squared Error (MSE): 0.3164194877895337
R-squared (R^2) score: 0.6991313738653087


In [36]:
import numpy as np

def mcrmse(y_true, y_pred):
    rmse_per_column = np.sqrt(np.mean((y_true - y_pred)**2, axis=0))
    mcrmse_value = np.mean(rmse_per_column)
    return mcrmse_value

mcrmse_score = mcrmse(y_test, y_pred)
print("Mean Columnwise Root Mean Squared Error (MCRMSE):", mcrmse_score)


Mean Columnwise Root Mean Squared Error (MCRMSE): 0.5574291357675863


Using MLP regressor on embeddings.

In [48]:
from sklearn.neural_network import MLPRegressor
y = merged_df[['content', 'wording']]
X_train, X_test, y_train, y_test = train_test_split(X_text_embeddings, y, test_size=0.2, random_state=42)
model = MLPRegressor(random_state=0,hidden_layer_sizes=[384,120])

2024-05-11 16:49:32,022 - INFO - Using 5x5 cross validation


  0%|          | 0/25 [00:00<?, ?it/s]

AttributeError: 'numpy.ndarray' object has no attribute 'iloc'

In [49]:
X_train = pd.DataFrame(X_train)
cross_validate(model,X_train,y_train,5,5)

2024-05-11 16:50:59,950 - INFO - Using 5x5 cross validation


  0%|          | 0/25 [00:00<?, ?it/s]

Metric        rmse                                 mae            \
Target     content   wording mean_columnwise   content   wording   
mean      0.505169  0.667834        0.586502  0.393945  0.521099   
stdev     0.014642  0.013138         0.01389  0.012041  0.010104   
n_trials        25        25               2        25        25   

Metric                          r2                            
Target   mean_columnwise   content   wording mean_columnwise  
mean            0.457522  0.764228   0.58835        0.676289  
stdev           0.011072  0.017718  0.019163        0.018441  
n_trials               2        25        25               2

In [37]:
features_df = pd.read_csv('feature_df.csv')
merged_df_2 = pd.merge(merged_df, features_df, on=['student_id', 'prompt_id', 'text'], how='inner')

In [39]:
selected_features = features_df.drop(columns=['student_id', 'prompt_id', 'text', 'content', 'wording'])

selected_features_array = selected_features.values

X_combined = np.hstack((X_embeddings, selected_features_array))


X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)
model = MultiOutputRegressor(HistGradientBoostingRegressor(random_state=0))

for train_i, test_i in k_folds.split(X_train):
    fold_train_X, fold_test_X = X_train[train_i], X_train[test_i]
    fold_train_y, fold_test_y = y_train.to_numpy()[train_i], y_train.to_numpy()[test_i]
    model.fit(fold_train_X, fold_train_y)


Mean Columnwise Root Mean Squared Error (MCRMSE): 0.5574291357675863


In [40]:
y_pred = model.predict(X_test)
mcrmse_score = mcrmse(y_test, y_pred)
print("Mean Columnwise Root Mean Squared Error (MCRMSE):", mcrmse_score)

Mean Columnwise Root Mean Squared Error (MCRMSE): 0.5046580605839339


In [44]:
X_train = pd.DataFrame(X_train)
cross_validate(model, X_train, y_train, 5, 5)

2024-05-11 15:23:39,363 - INFO - Using 5x5 cross validation


  0%|          | 0/25 [00:00<?, ?it/s]

Metric        rmse                                 mae            \
Target     content   wording mean_columnwise   content   wording   
mean      0.429815   0.57985        0.504833  0.325089   0.44464   
stdev     0.011478  0.015476        0.013477  0.007346  0.010607   
n_trials        25        25               2        25        25   

Metric                          r2                            
Target   mean_columnwise   content   wording mean_columnwise  
mean            0.384864  0.829466  0.689828        0.759647  
stdev           0.008976  0.010188  0.013033        0.011611  
n_trials               2        25        25               2

In [46]:
joe_features_df = pd.read_csv('joe_features.csv')

selected_joe_features = joe_features_df[['mean word length', 'stopwords_count', 'repeated_words_prompt_text', 'repeated_words_prompt_question', 'fleschReadingEase']]

selected_joe_features_array = selected_joe_features.values

X_combined_with_joe_features = np.hstack((X_combined, selected_joe_features_array))

X_train, X_test, y_train, y_test = train_test_split(X_combined_with_joe_features, y, test_size=0.2, random_state=42)
model = MultiOutputRegressor(HistGradientBoostingRegressor(random_state=0))

for train_i, test_i in k_folds.split(X_train):
    fold_train_X, fold_test_X = X_train[train_i], X_train[test_i]
    fold_train_y, fold_test_y = y_train.to_numpy()[train_i], y_train.to_numpy()[test_i]
    model.fit(fold_train_X, fold_train_y)

In [47]:
y_pred = model.predict(X_test)
mcrmse_score = mcrmse(y_test, y_pred)
print("Mean Columnwise Root Mean Squared Error (MCRMSE):", mcrmse_score)
X_train = pd.DataFrame(X_train)
cross_validate(model, X_train, y_train, 5, 5)

2024-05-11 16:08:52,141 - INFO - Using 5x5 cross validation


Mean Columnwise Root Mean Squared Error (MCRMSE): 0.49645004388739866


  0%|          | 0/25 [00:00<?, ?it/s]

Metric        rmse                                 mae            \
Target     content   wording mean_columnwise   content   wording   
mean      0.428294    0.5756        0.501947  0.324224  0.440986   
stdev     0.011868  0.015627        0.013747  0.007379  0.010543   
n_trials        25        25               2        25        25   

Metric                          r2                            
Target   mean_columnwise   content   wording mean_columnwise  
mean            0.382605  0.830628  0.694359        0.762494  
stdev           0.008961  0.010949  0.013077        0.012013  
n_trials               2        25        25               2